In [1]:
import pandas as pd
import numpy as np
from scipy.stats import bernoulli, binom, norm, poisson,\
nbinom, geom, hypergeom, expon, gamma

### 특수한 이산형 확률분포

- 베르누이 분포

In [2]:
# 기댓값과 분산 계산하기
p = 0.3
q = 1-p
E = p
V = p*q # 분산
print(f'[수기] 해당 분포 기댓값은 {E:.3f}, 분산은 {V:.3f}')

E, V = bernoulli.stats(p)
print(f'[라이브러리] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')

[수기] 해당 분포 기댓값은 0.300, 분산은 0.210
[라이브러리] 해당 분포의 기댓값은 0.300 분산은 0.210


- 이항분포

In [3]:
# combination 함수 정의하기
fac = np.math.factorial
def combi(a, b) : 
    result = fac(a)/fac(b)/fac(a-b)
    return result

# 근로자가 내년에 회사에서 일하지 않을 확률이 0.1이라고 추정한 경우,
# 시간제 근로자 3명을 무작위로 뽑았을 때 그 중 한 명이 금년에 회사를 떠날 확률은?
n = 3  # 샘플 개수
x = 1  # 확률 변수(떠날 샘플 개수)
p = 0.1

# 기댓값과 분산 계산하기
E = n*p
v = n*p*(1-p)
print(f'[수기] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')
E2, V2 = binom.stats(n, p)
print(f'[라이브러리] 해당 분포의 기댓값은 {E2:.3f} 분산은 {V2:.3f}', '\n')

# 확률질량함수 계산하기
pmf = combi(n, x) * p**x * (1-p)**(n-x)
print('[수기] 해당 분포의 확률질량함수(pmf): %.3f' % (pmf))
print('[라이브러리] 해당 분포의 확률질량함수(pmf): ', binom.pmf(x, n, p), '\n')

# 누적확률질량함수 계산하기: 1명 이하로 떠날 확률은?
cdf = 0
for i in range(0, x+1) : 
    cdf += binom.pmf(i, n, p)
print('[수기] 해당 분포의 누적확률질량함수(cdf): ', cdf)
print('[라이브러리] 해당 분포의 누적확률질럄함수(cdf): ', binom.cdf(x, n, p))

[수기] 해당 분포의 기댓값은 0.300 분산은 0.210
[라이브러리] 해당 분포의 기댓값은 0.300 분산은 0.270 

[수기] 해당 분포의 확률질량함수(pmf): 0.243
[라이브러리] 해당 분포의 확률질량함수(pmf):  0.243 

[수기] 해당 분포의 누적확률질량함수(cdf):  0.9720000000000001
[라이브러리] 해당 분포의 누적확률질럄함수(cdf):  0.972


In [4]:
# 이항분포의 정규분포 근사
x = 10
n = 30
p = 0.5

b_result = binom.pmf(x, n, p)
E = n*p
sigma = np.sqrt(E*(1-p))
zstat1 = (x-0.5-E)/sigma # 이항분포의 x에서 -0.5
zstat2 = (x+0.5-E)/sigma # 이항분포의 x에서 +0.5
n_result = norm.cdf(zstat2) - norm.cdf(zstat1)

print(f'이항분포로 계산하면 {b_result:.3f} 정규분포로 계산하면 {n_result:.3f}')

# 이항분포의 포아송분포 근사
x = 10
n = 100
p = 0.05
E = n*p

b_result = binom.pmf(x, n, p)
p_result = poisson.pmf(x, E)
print(f'이항분포로 계산하면 {b_result:.3f} 포아송분포로 근사계산하면 {p_result:.3f}')

이항분포로 계산하면 0.028 정규분포로 계산하면 0.028
이항분포로 계산하면 0.017 포아송분포로 근사계산하면 0.018


- 음이항분포

In [5]:
# 음이항분포
# A가 승리할 확률이 0.3이라면 5번째 경기에서 2번째로 이길 확률은?
fac = np.math.factorial
def combi(a, b) : 
    result = fac(a)/fac(b)/fac(a-b)
    return result

n = 5
k = 2
p = 0.3
q = 1-p

case = 2
if case == 1 :# 확률변수 X가 k번 성공할 때까지의 시행횟수가 x인 경우
    x = n
    E = k/p
    V = k*q/p**2
    pmf = combi(x-1, k-1) * p**k * q**(x-k)
    E2, V2 = np.nan, np.nan
    pkg_pmf, pkg_cdf = np.nan, np.nan
    cdf = 0
    for i in range(k, x+1) : 
        cdf += combi(i-1, k-1) * p**k * q**(i-k)
elif case == 2 : # 확률변수 X가 k번 성공할 때까지의 실패횟수 x인 경우
    x = n-k
    E = k*q / p
    V = k*q / p**2
    pmf = combi(x+k-1, k-1) * p**k * q**x
    E2, V2 = nbinom.stats(k, p)
    pkg_pmf, pkg_cdf = nbinom.pmf(x, k, p), nbinom.cdf(x, k, p)
    cdf = 0
    for i in range(0, x+1) : 
        cdf += nbinom.pmf(i, k, p)

# 기댓값과 분산 계산하기
print(f'[수기] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')
print(f'[라이브러리] 해당 분포의 기댓값은 {E2:.3f} 분산은 {V2:.3f}')

# 확률질량함수 계산하기
print('[수기] 확률질량함수(pmf): ', pmf)
print('[라이브러리] 확률질량함수(pmf): ', pkg_pmf, '\n')

# 누적확률질량함수 계산하기: 2번째 이하로 이길 확률은?
print('[수기] 누적확률질량함수(cdf): ', cdf)
print('[라이브러리] 누적확률질량함수(cdf): ', pkg_cdf)

[수기] 해당 분포의 기댓값은 4.667 분산은 15.556
[라이브러리] 해당 분포의 기댓값은 4.667 분산은 15.556
[수기] 확률질량함수(pmf):  0.12347999999999996
[라이브러리] 확률질량함수(pmf):  0.12348 

[수기] 누적확률질량함수(cdf):  0.47177999999999987
[라이브러리] 누적확률질량함수(cdf):  0.47178


-  기하분포

In [6]:
# 하나의 주사위를 세 번 던질 때 세 번째 시행에서 앞면 숫자가 6이 나올 확률?
n = 3
p = 1/6
q = 1-p

case = 1
if case == 1 : # 확률변수 X가 처음으로 성공할 때까지의 시행횟수 x인 경우
    x = n
    E = 1/p
    V = q/p**2
    pmf = q**(x-1)*p
    E2, V2 = geom.stats(p, moments='mv') # mean, variance
    pkg_pmf, pkg_cdf = geom.pmf(x, p), geom.cdf(x, p)
    cdf = 0
    for i in range(1, x+1) : 
        cdf += q**(i-1)*p
elif case == 2 : # 확률변수 X가 처음으로 성공할 때까지의 실패횟수 x인 경우
    x = n-1
    E = q/p
    V = q/p**2
    E2, V2 = np.nan, np.nan
    pmf = q**x * p
    pkg_pmf, pkg_cdf = np.nan, np.nan
    cdf = 0
    for i in range(0, x+1) : 
        cdf += q**i * p 

# 기댓값과 분산 계산하기
print(f'[수기] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')
print(f'[라이브러리] 해당 분포의 기댓값은 {E2:.3f} 분산은 {V2:.3f}', '\n')

# 확률질량함수 계산하기
print('[수기] 확률질량함수(pmf): ', pmf)
print('[라이브러리] 확률질량함수(pmf): ', pkg_pmf, '\n')

# 누적확률질량함수 계산하기: 2번째 이하로 이길 확률은?
print('[수기] 누적확률질량함수(cdf): ', cdf)
print('[라이브러리] 누적확률질량함수(cdf): ', pkg_cdf)

[수기] 해당 분포의 기댓값은 6.000 분산은 30.000
[라이브러리] 해당 분포의 기댓값은 6.000 분산은 30.000 

[수기] 확률질량함수(pmf):  0.11574074074074076
[라이브러리] 확률질량함수(pmf):  0.11574074074074076 

[수기] 누적확률질량함수(cdf):  0.42129629629629634
[라이브러리] 누적확률질량함수(cdf):  0.4212962962962962


- 초기하분포

In [7]:
# combination 함수 정의
fac = np.math.factorial
def combi(a, b) : 
    result = fac(a)/fac(b)/fac(a-b)
    return result

# 상자 속에 빨간 공이 90개, 파란 공이 10개 들어 있다. 임의로 1개씩 두 번 꺼내고
# 다시 넣지 않을 때 1개가 파란 공이 될 확률은?
N = 100
k = 10   # 성공요소(총 파란공) 개수
n = 2
x = 1    # 확인하고자 하는 사건(파란공 1개)-확률변수
p = k/N  # 모비율

# 기댓값과 분산 계산하기
E = n*p
V = n*p*(1-p)*((N-n)/(N-1))
print(f'[수기] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')
E2, V2 = hypergeom.stats(N, k, n)
print(f'[라이브러리] 해당 분포의 기댓값은 {E2:.3f} 분산은 {V2:.3f}', '\n')

# 확률질량함수 계산하기
pmf = combi(k,x) * combi(N-k, n-x)/combi(N,n)
print('[수기] 해당 분포의 확률질량함수(pmf): %.3f' % (pmf))
print('[라이브러리] 해당 분포의 확률질량함수(pmf): %.3f' % (hypergeom.pmf(x,N,k,n)), '\n')

# 누적확률질량함수 계산하기: 파란공을 1개 이하로 선택할 확률?
cdf = 0
for i in range(0, x+1) : 
    cdf += hypergeom.pmf(i, N, k, n)
print('[수기] 해당 분포의 누적확률질량함수(cdf): %.3f' % cdf)
print('[라이브러리] 해당 분포의 누적확률질량함수(cdf): %.3f' % (hypergeom.cdf(x,N,k,n)))

[수기] 해당 분포의 기댓값은 0.200 분산은 0.178
[라이브러리] 해당 분포의 기댓값은 0.200 분산은 0.178 

[수기] 해당 분포의 확률질량함수(pmf): 0.182
[라이브러리] 해당 분포의 확률질량함수(pmf): 0.182 

[수기] 해당 분포의 누적확률질량함수(cdf): 0.991
[라이브러리] 해당 분포의 누적확률질량함수(cdf): 0.991


In [8]:
# 초기하 분포의 이항분포 근사
b_result = binom.pmf(x, n, k/N)
h_result = hypergeom.pmf(x, N, k, n)
print(f'초기하분포로 계산하면 {h_result:.3f} 이항분포로 근사계산하면 {b_result:.3f}')

초기하분포로 계산하면 0.182 이항분포로 근사계산하면 0.180


- 포아송 분포

In [9]:
# 주말 저녁 시간 당 평균 6명이 응급실에 올 경우,
# 어떤 주말 저녁 30분 내 4명이 도착할 확률은?
x = 4    # 구간 내 사건 횟수(확률변수)
lam = 3  # 구간 내 평균

# 기댓값과 분산 계산하기
E = lam
V = lam
print(f'[수기] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')
E2, V2 = poisson.stats(lam, moments='mv')
print(f'[라이브러리] 해당 분포의 기댓값은 {E2:.3f} 분산은 {V2:.3f}', '\n')

# 확률질량함수 계산하기
pmf = lam**x * np.exp(-lam) / np.math.factorial(x)
print('[수기] 해당 분포의 확률질량함수(pmf): %.3f' % (pmf))
print('[라이브러리] 해당 분포의 확률질량함수(pmf): %.3f' % (poisson.pmf(x, lam)), '\n')

# 누적확률질량함수 계산하기: 4명 이하로 도착할 확률은?
cdf = 0
for i in range(0, x+1) : 
    cdf += poisson.pmf(i, lam)
print('[수기] 해당 분포의 누적확률질량함수(cdf): %.3f' % cdf)
print('[라이브러리] 해당 분포의 누적확률질량함수(cdf): %.3f' % (poisson.cdf(x, lam)))

[수기] 해당 분포의 기댓값은 3.000 분산은 3.000
[라이브러리] 해당 분포의 기댓값은 3.000 분산은 3.000 

[수기] 해당 분포의 확률질량함수(pmf): 0.168
[라이브러리] 해당 분포의 확률질량함수(pmf): 0.168 

[수기] 해당 분포의 누적확률질량함수(cdf): 0.815
[라이브러리] 해당 분포의 누적확률질량함수(cdf): 0.815


In [10]:
# 포아송 분포의 정규분포 근사
x = 10
lam = 20
E = V = lam

sigma = np.sqrt(V)
zstat1 = (x-0.5-lam)/sigma
zstat2 = (x+0.5-lam)/sigma
n_result = norm.cdf(zstat2) - norm.cdf(zstat1)
p_result = poisson.pmf(x, lam)

print(f'포아송 분포로 계산하면 {p_result:.3f} 정규분포로 계산하면 {n_result:.3f}')

포아송 분포로 계산하면 0.006 정규분포로 계산하면 0.007


### 특수한 연속형 확률분포

- 균일 분포

In [11]:
# 확률변수 X가 (5, 15)에서 균일분포를 따를 때 12와 15 사이의 확률은?
a = 5
b = 15
range_ = [(12, 15)]
p = 1 / (b-a)
cdf = 0
for (x1, x2) in range_ : 
    cdf += (x2-x1) * p  # 누적분포함수

E = (a+b)/2
V = (b-a)**2 / 12
print(f'[수기] 범위 내 누적확률은 {cdf:.3f}')
print(f'[수기] 해당 분포의 기대값은 {E:.3f} 분산은 {V:.3f}', '\n')

# 버스가 오전 7시부터 115분 간격으로 정류장을 출발한다. 한 승객이 이 정류장에
# 도착하는 시간은 7시에서 7시 30분 사이에 균등분포를 따른다고 할 때 이 승객이 버스를
# 5분 미만 기다릴 확률은?
a = 0
b = 30

# 승객이 버스를 5분 미만으로 기다리는 구간
# 7시 10분~15분, 7시 25분~30분
range_ = [(10,15), (25,30)]  # 구할 범위
p = 1/(b-a)
cdf = 0
for (x1,x2) in range_ : 
    cdf += (x2-x1) * p

E = (a+b)/2
V = (b-a)**2 / 12

print(f'[수기] 범위 내 누적확률은 {cdf:.3f}')
print(f'[수기] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')

[수기] 범위 내 누적확률은 0.300
[수기] 해당 분포의 기대값은 10.000 분산은 8.333 

[수기] 범위 내 누적확률은 0.333
[수기] 해당 분포의 기댓값은 15.000 분산은 75.000


- 정규분포

In [12]:
# 확률변수 X가 정규분포 N(30, 64)를 따를 때 26~46 구간의 확률은?
E = 30
V = 64
S = np.sqrt(V)
x1 = 26
x2 = 46
zstat1 = (x1-E)/S  # 표준화
zstat2 = (x2-E)/S  # 표준화
cdf = norm.cdf(zstat2) - norm.cdf(zstat1)

print(f'[라이브러리] 범위 내 누적롹률은 {cdf:.3f}')
print(f'[라이브러리] 해당 분포의 기댓값은{E:.3f} 분산은 {V:.3f}', '\n')

[라이브러리] 범위 내 누적롹률은 0.669
[라이브러리] 해당 분포의 기댓값은30.000 분산은 64.000 



- 지수분포

In [13]:
# 자동차들 사이 시간 간격이 평균 3분인 지수확률 분포를 따르는 경우
# 연속한 두 대의 차량이 도착하는 시간이  2분 이내일 확률은?
lam = 1/3  # 3분 동안 1건, 1분 동안 1/3건
x = 2      # 사건이 일어날 때까지 걸린 시간

E = 1/lam
V = 1/(lam**2)
# scale에 1/lam을 입력해야 함.
cdf = expon.cdf(x, scale = 1/lam)

print(f'[라이브러리] 누적확률은 {cdf:.3f}')
print(f'[라이브러리] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')

[라이브러리] 누적확률은 0.487
[라이브러리] 해당 분포의 기댓값은 3.000 분산은 9.000


- 감마분포: 지수분포의 일반화된 형태

In [14]:
# 낚시를 하는 데 어부가 물고기를 30분에 한 마리씩 잡는다. 어부가 4마리의 물고기를 
# 잡을 때까지 걸리는 시간이 2시간에서 4시간 사이로 소요될 확률은?
lam = 2   # lamda: 30분에 한 마리 = 1시간에 두 마리
beta = 1/lam
alpha = 4 # 4마리 물고기
E = alpha * beta
V = alpha * (beta**2)
range_ = (2, 4)
# scale에 beta를 입력해야 함.
cdf = gamma.cdf(range_[1], alpha, scale=beta) - gamma.cdf(range_[0], alpha, scale=beta)
print(f'[라이브러리] 누적확률은 {cdf:.3f}')
print(f'[라이브러리] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')

[라이브러리] 누적확률은 0.391
[라이브러리] 해당 분포의 기댓값은 2.000 분산은 1.000


In [15]:
'''
배송 시간이 alpha = 20, lambda = 1.6인 감마분포를 따를 때, 
20개 철판을 배송할 때 걸리는 시간이 15분 이내일 확률은? 
'''
lam = 1.6
beta = 1 / lam
alpha = 20
E = alpha * beta
V = alpha * (beta**2)
range_ = (0, 15)
# scale에 beta를 입력해야 함.
cdf = gamma.cdf(range_[1], alpha, scale=beta) - gamma.cdf(range_[0], alpha, scale=beta)

print(f'[라이브러리] 누적확률은 {cdf:.3f}')
print(f'[라이브러리] 해당 분포의 기댓값은 {E:.3f} 분산은 {V:.3f}')

[라이브러리] 누적확률은 0.820
[라이브러리] 해당 분포의 기댓값은 12.500 분산은 7.812
